In [ ]:
import lmdb
import json
import cv2
import math
import numpy as np
np.set_printoptions(precision=4,suppress=True)

In [ ]:
rr_debug = True

if rr_debug:
    import rerun as rr
else:
    class FakeRR(object):
        def init(*args, **kwargs):
            pass
        def __getattr__(self, key):
            def nothing(*args, **kwargs):
                return None
            return nothing
    rr = FakeRR()

In [ ]:
def objp():
    ret  = []
    for i in range(chessboard_size[0]):
        for j in range(chessboard_size[1]):
            ret.append((j*0.001, i*0.001, 0))
    return np.array(ret)

def distance_to_line(point, line_start, line_end):
    x1, y1 = line_start[0]
    x2, y2 = line_end[0]
    x0, y0 = point[0]

    numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    denominator = math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)

    distance = numerator / denominator
    return distance

def reorder_corners(chessboard_size, corners):
    assert len(corners) == chessboard_size[0] * chessboard_size[1], "点数量对不上"
    
    norm_to_center = corners - np.average(corners, axis=0)

    # 找四个角点
    edges = {}

    for (dis, idx) in sorted([(n[0], idx) for idx, n in enumerate(np.linalg.norm(norm_to_center, axis=2))], reverse=True):
        p = norm_to_center[idx]
        key = tuple([int(i) for i in np.sign(p)[0]])

        if key in edges:
            continue
        else:
            edges[key] = (p, idx)

        if len(edges) == 4:
            break
    
    cube_len = np.linalg.norm(edges[(1, 1)][0] - edges[(-1, -1)][0]) / (chessboard_size[0] - 1)
    
    # 找左右两列
    left_col = []
    for idx, p in enumerate(norm_to_center):
        dis = distance_to_line(p, edges[(-1, -1)][0], edges[(-1, 1)][0])
        if dis < cube_len * 0.5:
            left_col.append((p[0][1], idx))

    assert len(left_col) == chessboard_size[1], "左侧列没找全或多了"
    left_col.sort()

    right_col = []
    for idx, p in enumerate(norm_to_center):
        dis = distance_to_line(p, edges[(1, -1)][0], edges[(1, 1)][0])
        if dis < cube_len * 0.5:
            right_col.append((p[0][1], idx))

    assert len(right_col) == chessboard_size[1], "右侧列没找全或多了"
    right_col.sort()

    all_idx = set(range(chessboard_size[0] * chessboard_size[1]))
    ordered_seq = []

    # 找到每一行
    for row_idx in range(chessboard_size[1]):
        left = norm_to_center[left_col[row_idx][1]]
        right = norm_to_center[right_col[row_idx][1]]

        this_row = []
        for idx in all_idx:
            p = norm_to_center[idx]
            dis = distance_to_line(p, left, right)
            if dis < cube_len * 0.5:
                this_row.append((p[0][0], idx))
        
        assert len(this_row) == chessboard_size[0], f"第 {row_idx} 行没找全或多了"
        this_row.sort()

        for _, idx in this_row:
            ordered_seq.append(idx)
            all_idx.remove(idx)
    
    return np.array([corners[i] for i in ordered_seq])

# 相机参数计算

In [ ]:
cam_table = "v"

mdb_path = fr"D:\records\{cam_table}_cam"
pose_path = fr"E:\bitBucket\xyf_mj\ws\{cam_table}_all_pose.npy"
chessboard_size = (48, 48)

In [ ]:
rr.init("mdb_view", spawn=True)

use_count = 2
trigger_map = {}

imgpoints = []
img_size = None

env = lmdb.open(mdb_path)
with env.begin() as txn:
    meta = json.loads(txn.get(b"meta.json"))

    for img in meta["image_list"]:
        count_idx = trigger_map.get(img["trigger_id"], 0)
        trigger_map[img["trigger_id"]] = count_idx + 1

        if count_idx != use_count:
            continue

        png_data = txn.get(img["name"].encode())
        gray_img = cv2.imdecode(np.frombuffer(
            png_data, np.uint8), cv2.IMREAD_GRAYSCALE)

        res, bin_img = cv2.threshold(
            gray_img, 10, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        ret, corners = cv2.findChessboardCorners(
            bin_img, chessboard_size, flags=cv2.CALIB_CB_PLAIN )
        
        if corners is None:
            ret, corners = cv2.findChessboardCorners(
                bin_img, chessboard_size, flags=cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE )


        rr.set_time_seconds("ts", img["seconds"]+img["nanos"] * 1e-9)
        
        rr.log("img", rr.Image(gray_img), rr.AnyValues(**img))
        rr.log("bin_img", rr.Image(bin_img), rr.AnyValues(**img))

        if corners is not None:
            rr.log("corners", rr.Points2D(np.array(corners[:, 0, :])))

            try:
                criteria = (cv2.TERM_CRITERIA_EPS +
                            cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
                corners2 = cv2.cornerSubPix(
                    gray_img, corners, (10, 10), (-1, -1), criteria)
                corners3 = reorder_corners(chessboard_size, corners2)

               # rr.log("corners_dir", rr.LineStrips2D(np.array(corners2[:, 0, :])))

                
                rr.log("corners3", rr.Points2D(corners3[:, 0, :]))
                rr.log("corners_dir", rr.LineStrips2D(np.array(corners3[:, 0, :])))

                imgpoints.append((img["trigger_id"], img["name"], corners3))
                img_size = gray_img.shape[:2]

            except Exception as e:
                print(e)
        else:
            print("No corners")
        
        # if img["trigger_id"] == 73:
        #     print("break")
        #     break
        # break
len(imgpoints)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50000000, 0.00001)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    np.array([objp() for _ in imgpoints], dtype=np.float32), 
    np.array([m for _, _, m in imgpoints], dtype=np.float32), 
    img_size[::-1], None, None, criteria=criteria, 
    #flags=cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2  | cv2.CALIB_FIX_K3  |\
    #        cv2.CALIB_FIX_K4  | cv2.CALIB_FIX_K5  | cv2.CALIB_FIX_K6 
            )

print(ret)
mtx,dist

In [ ]:
all_pose = np.load(pose_path)
all_pose.shape

In [ ]:
R_gripper2base = []
t_gripper2base = []
R_target2cam = []
t_target2cam = []


for idx, _, points in imgpoints:
    _, rvec, tvec = cv2.solvePnP(objp(), points, mtx, dist)
    rmat, _ = cv2.Rodrigues(rvec)

    R_target2cam.append(rmat)
    t_target2cam.append(tvec)

    rmat = all_pose[idx][:3,:3]
    tvec = all_pose[idx][:3,3]

    R_gripper2base.append(rmat)
    t_gripper2base.append(tvec)

if True: # eye_to_hand:
    # change coordinates from gripper2base to base2gripper
    R_base2gripper, t_base2gripper = [], []
    for R, t in zip(R_gripper2base, t_gripper2base):
        R_b2g = R.T
        t_b2g = -R_b2g @ t
        R_base2gripper.append(R_b2g)
        t_base2gripper.append(t_b2g)
    
    # change parameters values
    R_gripper2base = R_base2gripper
    t_gripper2base = t_base2gripper

handeye_R, handeye_t = cv2.calibrateHandEye(
    np.array(R_gripper2base, dtype=np.float32), 
    np.array(t_gripper2base, dtype=np.float32), 
    np.array(R_target2cam, dtype=np.float32), 
    np.array(t_target2cam, dtype=np.float32),
    method=cv2.CALIB_HAND_EYE_DANIILIDIS
)

In [ ]:
mtx, dist, handeye_R, handeye_t

In [ ]:
print(json.dumps({
    "width": 3072,
    "height": 2048,
    "mtx": mtx.tolist(),
    "dist": dist.tolist(),
    "handeye_R": handeye_R.tolist(),
    "handeye_t": handeye_t.tolist()
}, indent=2))